
# Perform Sentiment Analysis with scikit-learn(IMDb Reviews)

![](https://d1sjtleuqoc1be.cloudfront.net/wp-content/uploads/2019/04/25112909/shutterstock_1073953772.jpg)

**So I took a project based course on coursera,the topic being Sentiment Analysis with sckit-learn.**

**This kernel is the application of what I have learnt in the course.
**
**This is going to be a very short,easy but effective kernel!**

**I hope you too will learn something new!**

![source](https://cdn-images-1.medium.com/max/361/0*ga5rNPmVYBsCm-lz.)

* **Importing important libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.feature_extraction.text import CountVectorizer
count=CountVectorizer()


* **from sklearn.feature_extraction.text import CountVectorizer**:
Convert a collection of text documents to a matrix of token counts
This implementation produces a sparse representation of the counts using scipy.sparse.csr_matrix.

In [ ]:
data=pd.read_csv("../input/imdb-dataset-sentiment-analysis-in-csv-format/Train.csv")
data.head()

* **After reading the csv file which contains 40k movie reviews from IMDB,we see that there are two prominent columns.One being TEXT which contains the review and the other being LABEL which contains O's and 1's ,where 0-NEGATIVE and 1-POSITIVE REVIEW!!**

In [ ]:
data.shape


In [ ]:
fig=plt.figure(figsize=(5,5))
colors=["skyblue",'pink']
pos=data[data['label']==1]
neg=data[data['label']==0]
ck=[pos['label'].count(),neg['label'].count()]
legpie=plt.pie(ck,labels=["Positive","Negative"],
                 autopct ='%1.1f%%', 
                 shadow = True,
                 colors = colors,
                 startangle = 45,
                 explode=(0, 0.1))

* **Lets understand how Count Vectorizer works which will be applied later to the dataset!**

* **get_feature_names():Array mapping from feature integer indices to feature name.**

In [ ]:
df=["Hey Jude, refrain Dont carry the world upon your shoulders For well you know that its a fool Who plays it cool By making his world a little colder Na-na-na,a, na Na-na-na, na"]
bag=count.fit_transform(df)
print(count.get_feature_names())


* **Now from the array we can infer that conversion a collection of text documents to a matrix of token counts has been done below**

In [ ]:
print(bag.toarray())

* ** Next we are going to Import RE i.e Regular Expression Operation,we are using this library to remove html tags like '< a >'  or  <head/>.So whenever we come across these tage we replace them with an empty string ''.Next we will also be altering emojis/emoticons which can be smiley :) ,sad face :( or even some upset face :/.We will be shifting the emojis towards the end so that we can get a set of clean text.**

In [ ]:
import re
def preprocessor(text):
             text=re.sub('<[^>]*>','',text)
             emojis=re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
             text=re.sub('[\W]+',' ',text.lower()) +\
                ' '.join(emojis).replace('-','')
             return text   

* **So lets show you some examples to get some clarity incase you did not understand what has happened in the above code**

* **So lets call the preprocessing function and pass some text in it and observe the output to understand what exactly is happening when the function is called**

In [ ]:
preprocessor(data.loc[0,'text'][-50:])

* **In the below text that has been passed in the function we can see that it contains HTML tag and emojis.The text returned will not contain the html tag and the emojis will be pushed towards the end of the text!**

In [ ]:
preprocessor("<a> this is :(  aweomee wohhhh :)")

* **Now since we have seen the above examples we will be passing the text data from our train dataset to this preprocessor function to clean the data.**

In [ ]:
data['text']=data['text'].apply(preprocessor)

* **We are importing a new library PorterStemmer from nltk.stem.porter.It follows an algorithm for suffix stripping i.e it will bring the word to its base meaning like running will be changed to run ,eating will be changed to eat.This is for simplifying the data and removing unnecessary complexities in our text data**

In [ ]:
from nltk.stem.porter import PorterStemmer

porter=PorterStemmer()

* **So before we perform stemming we need to split the sentences into words.Again to make you understand this concept of splitting of text we will see some Examples.**

In [ ]:
def tokenizer(text):
        return text.split()

In [ ]:
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

* **Lets see what will happen to this text that we are passing in the tokenizer function.**

In [ ]:
tokenizer("Haters love Hating as they Hate")

* **In this next Example we will see the words being reduced to their base words,Haters will be changed to hater,hating is changed to hate**

In [ ]:
tokenizer_porter("Haters love Hating as they Hate")

* **Stopwords are the most common words in any natural language. For the purpose of analyzing text data and building NLP models, these stopwords might not add much value to the meaning of the document.Generally, the most common words used in a text are “the”, “is”, “in”, “for”, “where”, “when”, “to”, “at” etc.**

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stop=stopwords.words('english')

In [ ]:
from wordcloud import WordCloud
positivedata = data[ data['label'] == 1]
positivedata =positivedata['text']
negdata = data[data['label'] == 0]
negdata= negdata['text']

def wordcloud_draw(data, color = 'white'):
    words = ' '.join(data)
    cleaned_word = " ".join([word for word in words.split()
                              if(word!='movie' and word!='film')
                            ])
    wordcloud = WordCloud(stopwords=stop,
                      background_color=color,
                      width=2500,
                      height=2000
                     ).generate(cleaned_word)
    plt.figure(1,figsize=(10, 7))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()
    
print("Positive words are as follows")
wordcloud_draw(positivedata,'white')
print("Negative words are as follows")
wordcloud_draw(negdata)


* **Positive words that are highlighted are:love,great,perfect,good,beautiful,nice,excellent**

* **Negative words that are highlighted are:awful,waste,problem,stupid,horrible,bad,poor**

* **Surprisingly both have 'ONE' in common,i guess for the positve category reviews might be like "One of the best characters/movies/films I've watched and for the negative comments it might be like "I rate this movie one star or one of the worst directions/scripts etc**

* **from sklearn.feature_extraction.text import TfidfVectorizer:Convert a collection of raw documents to a matrix of TF-IDF features.TfidfVectorizer:-Transforms text to feature vectors that can be used as input to estimator.****

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf=TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None,tokenizer=tokenizer_porter,use_idf=True,norm='l2',smooth_idf=True)

* **Learn vocabulary and idf, return term-document matrix.This is equivalent to fit followed by transform, but more efficiently implemented.**

In [ ]:
y=data.label.values
x=tfidf.fit_transform(data.text)

* **from sklearn.model_selection import train_test_split:Split arrays/matrices into train and test subsets**

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(x,y,random_state=1,test_size=0.5,shuffle=False)

* **So now we will be using Logistic Regression for our Sentiment Analysis **

* **Logistic Regression is a Machine Learning classification algorithm that is used to predict the probability of a categorical dependent variable. In logistic regression, the dependent variable is a binary variable that contains data coded as 1 (yes, success, etc.) or 0 (no, failure, etc.).CV here stands for Cross Validation.The user can pass the number of folds as an argument cv of the function to perform k-fold cross-validation**

In [ ]:

from sklearn.linear_model import LogisticRegressionCV

clf=LogisticRegressionCV(cv=6,scoring='accuracy',random_state=0,n_jobs=-1,verbose=3,max_iter=500).fit(X_train,y_train)

y_pred = clf.predict(X_test)


* **So now lets print the Accuracy Score!!**

**Linear classifiers (SVM, logistic regression, a.o.) with SGD training.**

In [ ]:
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.linear_model import SGDClassifier
clf= SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=1000, tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, random_state=None, learning_rate='optimal', eta0=0.0, power_t=0.5, early_stopping=False, validation_fraction=0.1, n_iter_no_change=5, class_weight=None, warm_start=False, average=False)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))